In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.ERROR)
logging.root.setLevel('INFO')
import io
import pandas as pd
import geopandas as gpd
import numpy as np
import fiona as fio
import rasterio as rio
import shapely.geometry as sg
import requests
from pathlib import Path
import subprocess
import folium
import requests
import rasterio.plot
import matplotlib.pyplot as plt
import rasterio.plot
import descartes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path


CWD = Path('.').resolve()
OUT_PATH = CWD / '00_input_data'
OUT_PATH.mkdir(exist_ok=True)

import sys
sys.path.append(str(CWD.parent / 'python_lib'))
sys.path

DATA_PATH = CWD / '00_input_data'
TEST_CASE_PATH = Path('.').resolve()
SIM_PATH = TEST_CASE_PATH / '01_simulation_resolution_8m'
SIM_PATH.mkdir(exist_ok=True)
COMPUTATION_PATH = SIM_PATH / 'Computation'
COMPUTATION_PATH.mkdir(exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from branca.element import MacroElement, JavascriptLink, CssLink, Template, Figure
class Draw(MacroElement):

    def __init__(self,
                 position='topleft',
                 circle=False,
                 rectangle=True,
                 polyline=False,
                 polygon=False,
                 marker=False,
                 circlemarker=False,
                 controller_callback="onLeafletDraw",
                 allow_multiple=False,
                 delete_on_creation=True,
                 editable=False
                 ):
        super().__init__()
        self._name = 'DrawControl'
        self.position = position
        self.circle = circle
        self.rectangle = rectangle
        self.polyline = polyline
        self.polygon = polygon
        self.marker = marker
        self.circlemarker = circlemarker
        self.controller_callback = controller_callback
        self.allow_multiple = allow_multiple
        self.delete_on_creation = delete_on_creation
        self.editable = editable

        self._template = Template(u"""
            {% macro script(this, kwargs) %}
            // FeatureGroup is to store editable layers.
            var drawnItems = new L.featureGroup().addTo({{this._parent.get_name()}});
            var {{this.get_name()}} = new L.Control.Draw({
                "draw": {
                    position: '{{this.position}}',
                    circle: {{this.circle.__str__().lower()}},
                    rectangle: {{this.rectangle.__str__().lower()}},
                    marker: {{this.marker.__str__().lower()}},
                    polyline: {{this.polyline.__str__().lower()}},
                    polygon: {{this.polygon.__str__().lower()}},
                    circlemarker: {{this.circlemarker.__str__().lower()}},
                },
                {% if this.editable %}
                    "edit": {"featureGroup": drawnItems}
                {% else %}
                    "edit": false
                {% endif %}
                });
            {{this._parent.get_name()}}.addControl({{this.get_name()}});
            {{this._parent.get_name()}}.on('draw:created', function (event) {
                var layer = event.layer,
                  type = event.layerType,
                  coords;
                {% if not this.allow_multiple %}
                    drawnItems.clearLayers();
                {% endif %}
                var coords = "shapely.geometry.shape(" + JSON.stringify(layer.toGeoJSON().geometry) + ")";
                {% if not this.delete_on_creation %}
                    drawnItems.addLayer(layer);
                {% else %}
                prompt('GeoJSON:', coords);
                {% endif %}              
                layer.on('click', function() {
                    prompt('GeoJSON:', coords);
                });
                
             });
            {% endmacro %}
            """)

    def render(self, **kwargs):
        super().render()

        figure = self.get_root()
        assert isinstance(figure, Figure), ('You cannot render this Element '
                                            'if it is not in a Figure.')

        figure.header.add_child(
            JavascriptLink('https://cdn.rawgit.com/Leaflet/Leaflet.draw/v0.4.12/dist/leaflet.draw.js'))  # noqa
        figure.header.add_child(
            CssLink('https://cdn.rawgit.com/Leaflet/Leaflet.draw/v0.4.12/dist/leaflet.draw.css'))  # noqa

In [11]:
CWD.parent.parent / 'GRAMM'

WindowsPath('C:/Users/nabil/GRAMM')

In [13]:
TEST = 0
dem_fname = DATA_PATH / 'rastert_reclass1.txt'
assert dem_fname.is_file()

In [14]:
with rio.open(DATA_PATH / 'rastert_reclass1.txt') as src:
    RCRS = src.crs
RCRS

In [15]:
with rio.open(dem_fname) as src:
    if src.crs is not None:
        assert src.crs == RCRS
    dem_res = src.res
    dem_transform = src.transform   
    dem = src.read(1, masked=True)
    dem_extent = rio.plot.plotting_extent(dem, transform=dem_transform)
    dem_bbox = rio.coords.BoundingBox(*src.bounds)
dem_bbox

BoundingBox(left=11855525.769483635, bottom=-753150.7633052344, right=11855526.019483635, top=-753150.5133052344)

In [11]:
# 2. DEM Data Information

In [8]:
from osgeo import gdal

# Open file DEM
dem_file = 'DEMNAS_Pongkor.tif'
dem_dataset = gdal.Open(dem_file)

# Get Information of Number Cells (jumlah sel) from DEM dataset
num_cells_x = dem_dataset.RasterXSize
num_cells_y = dem_dataset.RasterYSize
num_cells_z = dem_dataset.RasterCount

print("Number of cells in x-direction:", num_cells_x)
print("Number of cells in y-direction:", num_cells_y)
print("Number of cells in z-direction:", num_cells_z)


ModuleNotFoundError: No module named 'osgeo'

# 1. Developing Mesh GRAMM

In [2]:
resolution = 8
grading = 1.025
DO_REF = True
nz = 35
dz0 = 4.0
ddz = 1.14
n_constant_cells = 4
    
    
resolution, grading, DO_REF, dz0, ddz

(8, 1.025, True, 4.0, 1.14)

In [3]:
#mesh adalah GRAL Domain
#ref adalah GRAMM domain

import shapely
import geopandas as gpd

# Definisi CRS yang diinginkan
RCRS = 3857  # Contoh EPSG untuk CRS yang diinginkan

# Buat objek geometri ref_bbox
ref_bbox = shapely.geometry.shape({
    "type": "Polygon",
    "coordinates": [[[106.518395, -6.588055], [106.622905, -6.588055], [106.622905, -6.656847], [106.518395, -6.656847]]
    ]
})

# Konversi objek geometri ke GeoSeries dan transformasi CRS
ref_bbox = gpd.GeoSeries(ref_bbox, crs="EPSG:4326").to_crs(epsg=RCRS).squeeze()

# Buat objek geometri mesh_bbox
mesh_bbox = shapely.geometry.shape({
    "type": "Polygon",
    "coordinates": [
        [[106.500535, -6.500743], [106.749906, -6.500743], [106.749906, -6.749451], [106.500535, -6.748129]]
    ]
})

resolution = 0.001  # Define the resolution (adjust the value as needed)

# Konversi objek geometri ke GeoSeries, transformasi CRS, dan lakukan buffering
mesh_bbox = gpd.GeoSeries(mesh_bbox, crs="EPSG:4326").to_crs(epsg=RCRS).buffer(1_000).buffer(-resolution * 2).squeeze()

# Cetak hasil mesh_bbox
print(mesh_bbox)

# Import library generate_xys
from mesher import generate_xys

# Lakukan pemanggilan fungsi generate_xys dengan parameter yang sesuai
pts_df, nx, ny, xs, ys, dist_x, dist_y, mesh_bbox, ref_bbox = generate_xys(
    resolution, mesh_bbox, 
    refinement_zone=ref_bbox if DO_REF else None, 
    refinement_grading=grading
)

# Jika ref_bbox kosong, isi dengan mesh_bbox
if ref_bbox is None:
    ref_bbox = mesh_bbox

# Cetak nx, ny, mesh_bbox, dan ref_bbox
nx, ny, mesh_bbox, ref_bbox


POLYGON ((11854585.32741121 -725217.0307363248, 11854590.142677307 -725119.0137431977, 11854604.542094743 -725021.9407558879, 11854628.386991667 -724926.7465912647, 11854661.447728686 -724834.3480211786, 11854703.405912831 -724745.6348943564, 11854753.857463848 -724661.4615667197, 11854812.31650574 -724582.6386733969, 11854878.22004594 -724509.9253219865, 11854950.933397366 -724444.0217817718, 11855029.756290665 -724385.5627398975, 11855113.929618316 -724335.1111888725, 11855202.642745135 -724293.153004728, 11855295.041315228 -724260.0922677078, 11855390.235479845 -724236.2473707848, 11855487.308467153 -724221.8479533491, 11855585.325460307 -724217.0326872516, 11883345.178100722 -724217.0326872516, 11883443.195093876 -724221.8479533491, 11883540.268081184 -724236.2473707848, 11883635.462245801 -724260.0922677078, 11883727.860815894 -724293.153004728, 11883816.573942713 -724335.1111888725, 11883900.747270364 -724385.5627398975, 11883979.570163663 -724444.0217817718, 11884052.28351509 -7

ModuleNotFoundError: No module named 'mesher'

In [7]:
pip install mesher

  Using cached mesher-2.1.4.tar.gz (60 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached vtk-9.3.0-cp311-cp311-win_amd64.whl.metadata (5.3 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached GDAL-3.8.5.tar.gz (802 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached metis-0.2a5-py3-none-any.whl
  Using cached mpi4py-3.1.6-cp311-cp311-win_amd64.whl.metadata (8.0 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
Using cached cloudpickle-3.0.0-py3-none-any.whl (20 kB)
Using

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [38 lines of output]
  *** scikit-build-core 0.8.2 using CMake 3.29.2 (wheel)
  *** Configuring CMake...
  2024-04-18 01:11:09,618 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
  loading initial cache file C:\Users\nabil\AppData\Local\Temp\tmpfywg7z8n\build\CMakeInit.txt
  -- Building for: Visual Studio 17 2022
  -- Selecting Windows SDK version 10.0.22621.0 to target Windows 10.0.22631.
  -- The C compiler identification is MSVC 19.39.33523.0
  -- The CXX compiler identification is MSVC 19.39.33523.0
  -- Detecting C compiler ABI info
  -- Detecting C compiler ABI info - done
  -- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.39.33519/bin/Hostx64/x64/cl.exe - skipped
  -- Detecting C compile features
  -- Detecting C compile features - done
  -- Detecting CXX compiler ABI info
  -- Detecti